In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [3]:
loader = TextLoader('speech.txt')
docs = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200,chunk_overlap=30)
doc = text_splitter.split_documents(docs)

Created a chunk of size 372, which is longer than the specified 200
Created a chunk of size 444, which is longer than the specified 200
Created a chunk of size 460, which is longer than the specified 200
Created a chunk of size 584, which is longer than the specified 200
Created a chunk of size 328, which is longer than the specified 200
Created a chunk of size 402, which is longer than the specified 200


In [4]:
doc

[Document(metadata={'source': 'speech.txt'}, page_content='I have three visions for India. In 3000 years of our history people from all over the world have come and invaded us, captured our lands, conquered our minds. From Alexander onwards the Greeks, the Turks, the Moguls, the Portuguese, the British, the French, the Dutch, all of them came and looted us, took over what was ours. Yet we have not done this to any other nation.'),
 Document(metadata={'source': 'speech.txt'}, page_content='We have not conquered anyone. We have not grabbed their land, their culture and their history and tried to enforce our way of life on them. Why? Because we respect the freedom of others. That is why my FIRST VISION is that of FREEDOM. I believe that India got its first vision of this in 1857, when we started the war of Independence. It is this freedom that we must protect and nurture and build on. If we are not free, no one will respect us.'),
 Document(metadata={'source': 'speech.txt'}, page_content=

In [5]:
%pip install sentence_transformers
%pip install transformers
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
import os
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(doc,embeddings)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
db

In [7]:
query = "What is the numer of visions of the speaker"
d = db.similarity_search(query)
d

[Document(id='1352ad03-f77a-4f09-a288-025ccbcbf1c9', metadata={'source': 'speech.txt'}, page_content='I have three visions for India. In 3000 years of our history people from all over the world have come and invaded us, captured our lands, conquered our minds. From Alexander onwards the Greeks, the Turks, the Moguls, the Portuguese, the British, the French, the Dutch, all of them came and looted us, took over what was ours. Yet we have not done this to any other nation.'),
 Document(id='41ed1943-d2f6-4f63-8205-6e903f2f3ac4', metadata={'source': 'speech.txt'}, page_content='I have a THIRD VISION. India must stand up to the world. Because I believe that unless India stands up to the world, no one will respect us. Only strength respects strength. We must be strong not only as a military power but also as an economic power. Both must go hand-in-hand. My good fortune was to have worked with three great minds. Dr.Vikram Sarabhai, of the Dept. of Space, Professor Satish Dhawan, who succeeded 

RETRIEVER

In [8]:
retriever = db.as_retriever()
docs_retrieved = retriever.invoke(query)
docs_retrieved

[Document(id='1352ad03-f77a-4f09-a288-025ccbcbf1c9', metadata={'source': 'speech.txt'}, page_content='I have three visions for India. In 3000 years of our history people from all over the world have come and invaded us, captured our lands, conquered our minds. From Alexander onwards the Greeks, the Turks, the Moguls, the Portuguese, the British, the French, the Dutch, all of them came and looted us, took over what was ours. Yet we have not done this to any other nation.'),
 Document(id='41ed1943-d2f6-4f63-8205-6e903f2f3ac4', metadata={'source': 'speech.txt'}, page_content='I have a THIRD VISION. India must stand up to the world. Because I believe that unless India stands up to the world, no one will respect us. Only strength respects strength. We must be strong not only as a military power but also as an economic power. Both must go hand-in-hand. My good fortune was to have worked with three great minds. Dr.Vikram Sarabhai, of the Dept. of Space, Professor Satish Dhawan, who succeeded 

In [9]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='1352ad03-f77a-4f09-a288-025ccbcbf1c9', metadata={'source': 'speech.txt'}, page_content='I have three visions for India. In 3000 years of our history people from all over the world have come and invaded us, captured our lands, conquered our minds. From Alexander onwards the Greeks, the Turks, the Moguls, the Portuguese, the British, the French, the Dutch, all of them came and looted us, took over what was ours. Yet we have not done this to any other nation.'),
  1.5733886),
 (Document(id='41ed1943-d2f6-4f63-8205-6e903f2f3ac4', metadata={'source': 'speech.txt'}, page_content='I have a THIRD VISION. India must stand up to the world. Because I believe that unless India stands up to the world, no one will respect us. Only strength respects strength. We must be strong not only as a military power but also as an economic power. Both must go hand-in-hand. My good fortune was to have worked with three great minds. Dr.Vikram Sarabhai, of the Dept. of Space, Professor Satish Dhawan

In [10]:
embeddings_vector= embeddings.embed_query(query)

In [11]:
docs_score_with_vector_search = db.similarity_search_by_vector(embeddings_vector)

In [12]:
docs_score_with_vector_search

[Document(id='1352ad03-f77a-4f09-a288-025ccbcbf1c9', metadata={'source': 'speech.txt'}, page_content='I have three visions for India. In 3000 years of our history people from all over the world have come and invaded us, captured our lands, conquered our minds. From Alexander onwards the Greeks, the Turks, the Moguls, the Portuguese, the British, the French, the Dutch, all of them came and looted us, took over what was ours. Yet we have not done this to any other nation.'),
 Document(id='41ed1943-d2f6-4f63-8205-6e903f2f3ac4', metadata={'source': 'speech.txt'}, page_content='I have a THIRD VISION. India must stand up to the world. Because I believe that unless India stands up to the world, no one will respect us. Only strength respects strength. We must be strong not only as a military power but also as an economic power. Both must go hand-in-hand. My good fortune was to have worked with three great minds. Dr.Vikram Sarabhai, of the Dept. of Space, Professor Satish Dhawan, who succeeded 

In [13]:
db.save_local("faiss_index")

In [15]:
new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)

CHROMA DB

In [1]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
loader = TextLoader("speech.txt")
data = loader.load()
data


[Document(metadata={'source': 'speech.txt'}, page_content='I have three visions for India. In 3000 years of our history people from all over the world have come and invaded us, captured our lands, conquered our minds. From Alexander onwards the Greeks, the Turks, the Moguls, the Portuguese, the British, the French, the Dutch, all of them came and looted us, took over what was ours. Yet we have not done this to any other nation.\n\nWe have not conquered anyone. We have not grabbed their land, their culture and their history and tried to enforce our way of life on them. Why? Because we respect the freedom of others. That is why my FIRST VISION is that of FREEDOM. I believe that India got its first vision of this in 1857, when we started the war of Independence. It is this freedom that we must protect and nurture and build on. If we are not free, no one will respect us.\n\nWe have 10 percent growth rate in most areas. Our poverty levels are falling. Our achievements are being globally rec

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=0)
splits = text_splitter.split_documents(data)

In [5]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectordb = Chroma.from_documents(documents=splits,embedding=embeddings)
vectordb

In [6]:
query = "What are the different visions of the speaker?"
docs = vectordb.similarity_search(query)
docs[0].page_content

'I have a THIRD VISION. India must stand up to the world. Because I believe that unless India stands up to the world, no one will respect us. Only strength respects strength. We must be strong not only as a military power but also as an economic power. Both must go hand-in-hand. My good fortune was to have worked with three great minds. Dr.Vikram Sarabhai, of the Dept. of Space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material. I was lucky to have'

In [8]:
# Saving to the disk
vectordb=Chroma.from_documents(documents=splits,embedding=embeddings,persist_directory="./chroma_db")

In [9]:
db2 = Chroma(persist_directory="./chroma_db",embedding_function=embeddings)
docs2  =db2.similarity_search(query)
print(docs[0].page_content)

I have a THIRD VISION. India must stand up to the world. Because I believe that unless India stands up to the world, no one will respect us. Only strength respects strength. We must be strong not only as a military power but also as an economic power. Both must go hand-in-hand. My good fortune was to have worked with three great minds. Dr.Vikram Sarabhai, of the Dept. of Space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material. I was lucky to have


In [10]:
print(db2.similarity_search_with_score(query))

[(Document(id='0d585f90-d5c4-44c9-bd58-59bce232befe', metadata={'source': 'speech.txt'}, page_content='I have a THIRD VISION. India must stand up to the world. Because I believe that unless India stands up to the world, no one will respect us. Only strength respects strength. We must be strong not only as a military power but also as an economic power. Both must go hand-in-hand. My good fortune was to have worked with three great minds. Dr.Vikram Sarabhai, of the Dept. of Space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material. I was lucky to have'), 443.64642333984375), (Document(id='1fedf1d1-ce23-4be7-8869-f019069ea46a', metadata={'source': 'speech.txt'}, page_content='I have three visions for India. In 3000 years of our history people from all over the world have come and invaded us, captured our lands, conquered our minds. From Alexander onwards the Greeks, the Turks, the Moguls, the Portuguese, the British, the French, the Dutch, all of 

In [11]:
retriever = vectordb.as_retriever()
retriever.invoke(query)[0].page_content

'I have a THIRD VISION. India must stand up to the world. Because I believe that unless India stands up to the world, no one will respect us. Only strength respects strength. We must be strong not only as a military power but also as an economic power. Both must go hand-in-hand. My good fortune was to have worked with three great minds. Dr.Vikram Sarabhai, of the Dept. of Space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material. I was lucky to have'